## Fine-Tuning LLMs (Lllama-2)

**By the end of the notebook you will learn**
- how to fine-tune llama2 model using Q-Lora
- Combine trained adapters with base model and save the serialized model
- push the weights to GCS bucket (will be used in inference later)

**References**
- https://www.philschmid.de/instruction-tune-llama-2
- https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32


### Setup

In [ ]:
# install required libaries
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7 \
                huggingface_hub \
                wandb \
                python-dotenv


In [ ]:
import os
os.environ["WANDB_API_KEY"]="PLACE_YOUR_KEY"
os.environ["HUGGING_FACE_HUB_TOKEN"]="PLACE_YOUR_KEY"
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

In [ ]:
# login to hugging face to get the gated model weights
from huggingface_hub import login
# make sure you have set the environment variable HUGGING_FACE_HUB_TOKEN in .env
login(token=os.environ["HUGGING_FACE_HUB_TOKEN"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import wandb
# make sure you have set the environment variable WANDB_API_KEY in .env
_ = wandb.login()
os.environ["WANDB_PROJECT"] = "fine-tuning" # change the project name based on your need

In [ ]:
# import libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


### Setup Training Arguments

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-hf-v2"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    #report_to="tensorboard"
    report_to="wandb",  # enable logging to W&B
    run_name="llama-train-3"  # name of the W&B run (optional)
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Launch Training Job and Track the progress with W&B

In [ ]:
# launch training job
trainer.train()
# finish the w&b tracking
wandb.finish()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.249000
50,1.533700


### Persist Adapter Weights and combine with Base Model

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
!ls

In [ ]:
!ls -l --block-size=M llama-2-7b-hf-v2

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
# create merged model
model = PeftModel.from_pretrained(base_model, new_model)
merged_model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# persist merged model weights
merged_model.save_pretrained(new_model,safe_serialization=True)
tokenizer.save_pretrained(new_model)

In [ ]:
!ls -l --block-size=M llama-2-7b-hf-v2

total 12983M
-rw-r--r-- 1 root root    1M Jul 31 05:46 adapter_config.json
-rw-r--r-- 1 root root  129M Jul 31 05:46 adapter_model.bin
-rw-r--r-- 1 root root    1M Jul 31 05:48 config.json
-rw-r--r-- 1 root root    1M Jul 31 05:48 generation_config.json
-rw-r--r-- 1 root root 9515M Jul 31 05:49 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root 3339M Jul 31 05:49 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root    1M Jul 31 05:49 model.safetensors.index.json
-rw-r--r-- 1 root root    1M Jul 31 05:46 README.md
-rw-r--r-- 1 root root    1M Jul 31 05:49 special_tokens_map.json
-rw-r--r-- 1 root root    1M Jul 31 05:49 tokenizer_config.json
-rw-r--r-- 1 root root    2M Jul 31 05:49 tokenizer.json


### Push Weights to GCS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gcloud auth activate-service-account --key-file=drive/MyDrive/gcp-service-account.json

In [ ]:
!pip install -qqq gcsfs

In [ ]:
import gcsfs
def upload_to_gcs(src_dir: str, gcs_dst: str):
    fs = gcsfs.GCSFileSystem(project='<YOUR_PROJECT_ID>', token='drive/MyDrive/gcp-service-account.json')
    fs.put(src_dir, gcs_dst, recursive=True)

In [ ]:
source = os.path.join(os.path.curdir, new_model)
destination = f"<GCS_BUCKET>/{new_model}"
print(f"Uploading model weights to {destination}")
upload_to_gcs(source, destination)

Uploading model weights to gs://ps-llm-collection/llm/llama-2-7b-hf-v2
